In [1]:
#splitting the EEG data into 9 classes by their subject, and train them individually.
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
y_train_valid -= 769
y_test -= 769
person_train_valid = person_train_valid.astype(int)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# datatype = torch.float
# torch.cuda.device(device)
# print(device)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Temporal Convolution
        self.conv1 = nn.Conv2d(1, 40, (1, 40), stride = (1, 2))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(40)
        # ELU
        self.elu1 = nn.ELU()
        # Spatial Convolution
        self.conv2 = nn.Conv2d(40, 40, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(40)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(40 * 28, 4)
        # Softmax
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [8]:
cnn = CNN()

In [9]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [10]:
def X_reshape(X, index, y):#y is person_train_valid
    arr = np.array([])
    for i in range(len(X)):
        if(y[i][0] == index):
            if(len(arr) == 0):
                arr = np.expand_dims(X[i], axis=0)
            else:
                arr = np.concatenate((arr, np.expand_dims(X[i], axis=0)), axis=0)
    return arr


In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [12]:
for ind in range(9):
    X_train_valid_reshape = X_reshape(X_train_valid, ind, person_train_valid)
    y_train_valid_reshape = X_reshape(y_train_valid, ind, person_train_valid)
    batch_size = 500
    num_iters = 1
    num_epoch = 50
    epoch_loss = [0.0 for i in range(num_epoch)]
    for epoch in range(num_epoch):
        counter = 0
        for train_index, valid_index in kf.split(X_train_valid_reshape):
            X_train_cur = torch.tensor(X_train_valid_reshape[train_index].reshape(X_train_valid_reshape[train_index].shape[0], -1, 22, 1000), dtype=torch.float)
            X_valid_cur = torch.tensor(X_train_valid_reshape[valid_index].reshape(X_train_valid_reshape[valid_index].shape[0], -1, 22, 1000), dtype=torch.float)
            y_train_cur = torch.tensor(y_train_valid_reshape[train_index], dtype=torch.long)
            y_valid_cur = torch.tensor(y_train_valid_reshape[valid_index], dtype=torch.long)
            for it in np.arange(num_iters):
                index = np.random.choice(X_train_cur.shape[0], batch_size)
                X_batch = X_train_cur[index, :, :, :]
                y_batch = y_train_cur[index]
                optimizer.zero_grad()
                outputs = cnn(X_batch)

                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss[epoch] += loss

                counter += 1
            # Validation Accuracy for current K-fold
            output_val = cnn(X_valid_cur)
            prediction_val = torch.argmax(output_val, axis = 1)
            accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
            print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
        epoch_loss[epoch] /= counter 
        print('Current Subject: {}, Current Epoch: {}, Epoch Loss: {}'.format(ind, epoch, epoch_loss[epoch]))
    X_test = torch.tensor(X_test.reshape(X_test.shape[0], -1, 22, 1000), dtype= torch.float)
    y_test = torch.tensor(y_test, dtype = torch.long)
    output_test = cnn(X_test)
    prediction_test = torch.argmax(output_test, axis = 1)
    accuracy_test = float(prediction_test.eq(y_test).sum()) / float(y_test.shape[0])
    print('Current Subject: {}, Current Epoch: {}, Test Accuracy: {}'.format(ind, epoch, accuracy_test))


Current Epoch: 0, Validation Accuracy: 0.25
Current Epoch: 0, Validation Accuracy: 0.3559322033898305
Current Epoch: 0, Validation Accuracy: 0.559322033898305
Current Epoch: 0, Validation Accuracy: 0.5254237288135594
Current Subject: 0, Current Epoch: 0, Epoch Loss: 1.3553060293197632
Current Epoch: 1, Validation Accuracy: 0.4
Current Epoch: 1, Validation Accuracy: 0.6271186440677966
Current Epoch: 1, Validation Accuracy: 0.6101694915254238
Current Epoch: 1, Validation Accuracy: 0.7288135593220338
Current Subject: 0, Current Epoch: 1, Epoch Loss: 1.2169169187545776
Current Epoch: 2, Validation Accuracy: 0.5833333333333334
Current Epoch: 2, Validation Accuracy: 0.711864406779661
Current Epoch: 2, Validation Accuracy: 0.6610169491525424
Current Epoch: 2, Validation Accuracy: 0.7627118644067796
Current Subject: 0, Current Epoch: 2, Epoch Loss: 1.1415741443634033
Current Epoch: 3, Validation Accuracy: 0.6333333333333333
Current Epoch: 3, Validation Accuracy: 0.7627118644067796
Current Epoc

Current Epoch: 27, Validation Accuracy: 1.0
Current Subject: 0, Current Epoch: 27, Epoch Loss: 0.7755880355834961
Current Epoch: 28, Validation Accuracy: 0.9333333333333333
Current Epoch: 28, Validation Accuracy: 0.9830508474576272
Current Epoch: 28, Validation Accuracy: 0.9661016949152542
Current Epoch: 28, Validation Accuracy: 1.0
Current Subject: 0, Current Epoch: 28, Epoch Loss: 0.7785243988037109
Current Epoch: 29, Validation Accuracy: 0.9333333333333333
Current Epoch: 29, Validation Accuracy: 0.9830508474576272
Current Epoch: 29, Validation Accuracy: 0.9661016949152542
Current Epoch: 29, Validation Accuracy: 1.0
Current Subject: 0, Current Epoch: 29, Epoch Loss: 0.7749511003494263
Current Epoch: 30, Validation Accuracy: 0.9333333333333333
Current Epoch: 30, Validation Accuracy: 0.9830508474576272
Current Epoch: 30, Validation Accuracy: 0.9661016949152542
Current Epoch: 30, Validation Accuracy: 1.0
Current Subject: 0, Current Epoch: 30, Epoch Loss: 0.7740623950958252
Current Epoch

Current Epoch: 6, Validation Accuracy: 0.711864406779661
Current Epoch: 6, Validation Accuracy: 0.711864406779661
Current Epoch: 6, Validation Accuracy: 0.7627118644067796
Current Epoch: 6, Validation Accuracy: 0.7796610169491526
Current Subject: 1, Current Epoch: 6, Epoch Loss: 1.0162928104400635
Current Epoch: 7, Validation Accuracy: 0.7288135593220338
Current Epoch: 7, Validation Accuracy: 0.6610169491525424
Current Epoch: 7, Validation Accuracy: 0.8305084745762712
Current Epoch: 7, Validation Accuracy: 0.8135593220338984
Current Subject: 1, Current Epoch: 7, Epoch Loss: 0.9930009841918945
Current Epoch: 8, Validation Accuracy: 0.7457627118644068
Current Epoch: 8, Validation Accuracy: 0.6949152542372882
Current Epoch: 8, Validation Accuracy: 0.8305084745762712
Current Epoch: 8, Validation Accuracy: 0.7966101694915254
Current Subject: 1, Current Epoch: 8, Epoch Loss: 0.9666881561279297
Current Epoch: 9, Validation Accuracy: 0.7627118644067796
Current Epoch: 9, Validation Accuracy: 0.

Current Epoch: 33, Validation Accuracy: 0.8983050847457628
Current Epoch: 33, Validation Accuracy: 0.8983050847457628
Current Epoch: 33, Validation Accuracy: 0.9322033898305084
Current Epoch: 33, Validation Accuracy: 0.8813559322033898
Current Subject: 1, Current Epoch: 33, Epoch Loss: 0.8348464965820312
Current Epoch: 34, Validation Accuracy: 0.8983050847457628
Current Epoch: 34, Validation Accuracy: 0.8983050847457628
Current Epoch: 34, Validation Accuracy: 0.9322033898305084
Current Epoch: 34, Validation Accuracy: 0.8813559322033898
Current Subject: 1, Current Epoch: 34, Epoch Loss: 0.8316726088523865
Current Epoch: 35, Validation Accuracy: 0.8983050847457628
Current Epoch: 35, Validation Accuracy: 0.8983050847457628
Current Epoch: 35, Validation Accuracy: 0.9322033898305084
Current Epoch: 35, Validation Accuracy: 0.8813559322033898
Current Subject: 1, Current Epoch: 35, Epoch Loss: 0.8505306243896484
Current Epoch: 36, Validation Accuracy: 0.8983050847457628
Current Epoch: 36, Vali

/Users/yifan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/yifan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

Current Epoch: 16, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 16, Epoch Loss: 0.7871743440628052
Current Epoch: 17, Validation Accuracy: 0.95
Current Epoch: 17, Validation Accuracy: 0.95
Current Epoch: 17, Validation Accuracy: 0.9661016949152542
Current Epoch: 17, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 17, Epoch Loss: 0.7867755889892578
Current Epoch: 18, Validation Accuracy: 0.95
Current Epoch: 18, Validation Accuracy: 0.95
Current Epoch: 18, Validation Accuracy: 0.9830508474576272
Current Epoch: 18, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 18, Epoch Loss: 0.7923979759216309
Current Epoch: 19, Validation Accuracy: 0.95
Current Epoch: 19, Validation Accuracy: 0.95
Current Epoch: 19, Validation Accuracy: 0.9661016949152542
Current Epoch: 19, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 19, Epoch Loss: 0.7841749787330627
Current Epoch: 20, Validation Accurac

Current Epoch: 44, Validation Accuracy: 0.9833333333333333
Current Epoch: 44, Validation Accuracy: 0.9833333333333333
Current Epoch: 44, Validation Accuracy: 0.9830508474576272
Current Epoch: 44, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 44, Epoch Loss: 0.7574096918106079
Current Epoch: 45, Validation Accuracy: 0.9833333333333333
Current Epoch: 45, Validation Accuracy: 0.9833333333333333
Current Epoch: 45, Validation Accuracy: 0.9830508474576272
Current Epoch: 45, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 45, Epoch Loss: 0.7655273675918579
Current Epoch: 46, Validation Accuracy: 0.9833333333333333
Current Epoch: 46, Validation Accuracy: 0.9833333333333333
Current Epoch: 46, Validation Accuracy: 0.9830508474576272
Current Epoch: 46, Validation Accuracy: 0.9830508474576272
Current Subject: 6, Current Epoch: 46, Epoch Loss: 0.7628207802772522
Current Epoch: 47, Validation Accuracy: 0.9833333333333333
Current Epoch: 47, Vali

Current Epoch: 21, Validation Accuracy: 0.9310344827586207
Current Epoch: 21, Validation Accuracy: 0.9655172413793104
Current Epoch: 21, Validation Accuracy: 0.9655172413793104
Current Epoch: 21, Validation Accuracy: 0.9310344827586207
Current Subject: 7, Current Epoch: 21, Epoch Loss: 0.7897602319717407
Current Epoch: 22, Validation Accuracy: 0.9482758620689655
Current Epoch: 22, Validation Accuracy: 0.9827586206896551
Current Epoch: 22, Validation Accuracy: 0.9655172413793104
Current Epoch: 22, Validation Accuracy: 0.9310344827586207
Current Subject: 7, Current Epoch: 22, Epoch Loss: 0.7934240102767944
Current Epoch: 23, Validation Accuracy: 0.9482758620689655
Current Epoch: 23, Validation Accuracy: 0.9655172413793104
Current Epoch: 23, Validation Accuracy: 0.9655172413793104
Current Epoch: 23, Validation Accuracy: 0.9482758620689655
Current Subject: 7, Current Epoch: 23, Epoch Loss: 0.7846978902816772
Current Epoch: 24, Validation Accuracy: 0.9482758620689655
Current Epoch: 24, Vali

Current Epoch: 48, Validation Accuracy: 0.9655172413793104
Current Epoch: 48, Validation Accuracy: 0.9827586206896551
Current Epoch: 48, Validation Accuracy: 0.9827586206896551
Current Epoch: 48, Validation Accuracy: 0.9827586206896551
Current Subject: 7, Current Epoch: 48, Epoch Loss: 0.7645021677017212
Current Epoch: 49, Validation Accuracy: 0.9827586206896551
Current Epoch: 49, Validation Accuracy: 0.9827586206896551
Current Epoch: 49, Validation Accuracy: 0.9827586206896551
Current Epoch: 49, Validation Accuracy: 0.9827586206896551
Current Subject: 7, Current Epoch: 49, Epoch Loss: 0.7583749294281006
Current Subject: 7, Current Epoch: 49, Test Accuracy: 0.4650112866817156
Current Epoch: 0, Validation Accuracy: 0.6896551724137931
Current Epoch: 0, Validation Accuracy: 0.6551724137931034
Current Epoch: 0, Validation Accuracy: 0.7068965517241379
Current Epoch: 0, Validation Accuracy: 0.5789473684210527
Current Subject: 8, Current Epoch: 0, Epoch Loss: 1.115881323814392
Current Epoch: 

Current Epoch: 26, Validation Accuracy: 0.9827586206896551
Current Epoch: 26, Validation Accuracy: 1.0
Current Epoch: 26, Validation Accuracy: 0.9298245614035088
Current Subject: 8, Current Epoch: 26, Epoch Loss: 0.7661783695220947
Current Epoch: 27, Validation Accuracy: 1.0
Current Epoch: 27, Validation Accuracy: 1.0
Current Epoch: 27, Validation Accuracy: 1.0
Current Epoch: 27, Validation Accuracy: 0.9298245614035088
Current Subject: 8, Current Epoch: 27, Epoch Loss: 0.7606291770935059
Current Epoch: 28, Validation Accuracy: 1.0
Current Epoch: 28, Validation Accuracy: 1.0
Current Epoch: 28, Validation Accuracy: 1.0
Current Epoch: 28, Validation Accuracy: 0.9122807017543859
Current Subject: 8, Current Epoch: 28, Epoch Loss: 0.76345294713974
Current Epoch: 29, Validation Accuracy: 1.0
Current Epoch: 29, Validation Accuracy: 1.0
Current Epoch: 29, Validation Accuracy: 1.0
Current Epoch: 29, Validation Accuracy: 0.9298245614035088
Current Subject: 8, Current Epoch: 29, Epoch Loss: 0.7596

In [13]:
torch.save(cnn, 'model.pt')

/Users/yifan/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
